In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import os
from dotenv import load_dotenv
import pyodbc

import pyodbc
import requests

import pandas as pd 


import os
from dotenv import load_dotenv
import pyodbc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score , mean_squared_error, mean_absolute_error
import pickle

import mlflow


In [3]:
# Charger les variables d'environnement
load_dotenv()

server = os.getenv('DB_SERVER')
database = os.getenv('DB_DATABASE')
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')

driver= '{ODBC Driver 17 for SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

recuperation ded données BDD vers un DF 

In [4]:


# Créer un DataFrame pour chaque table
df_produit = pd.read_sql_query('SELECT * FROM Produit', cnxn)
df_caracteristiques = pd.read_sql_query('SELECT * FROM Caracteristiques', cnxn)
df_dimensions = pd.read_sql_query('SELECT * FROM Dimensions', cnxn)

# Fermer la connexion
cnxn.close()

C:\Users\impej\AppData\Local\Temp\ipykernel_27104\1192730980.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_produit = pd.read_sql_query('SELECT * FROM Produit', cnxn)
C:\Users\impej\AppData\Local\Temp\ipykernel_27104\1192730980.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_caracteristiques = pd.read_sql_query('SELECT * FROM Caracteristiques', cnxn)
C:\Users\impej\AppData\Local\Temp\ipykernel_27104\1192730980.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dimensions = pd.read_sql_query('SELECT * FROM

In [5]:
# Fusionner df_produit et df_caracteristiques sur 'ID_Produit'
df = pd.merge(df_produit, df_caracteristiques, on='ID_Produit')

# Fusionner le DataFrame résultant avec df_dimensions sur 'ID_Produit'
df = pd.merge(df, df_dimensions, on='ID_Produit')
df

,ID_Produit,URL_Produit,Prix,Info_generale,Descriptif,Note,Date_scrap,Marque,ID_Caracteristique,Consommation,...,Bruit,Saisonalite,Type_Vehicule,Runflat,ID_Dimension,Largeur,Hauteur,Diametre,Charge,Vitesse
0,33212,https://www.carter-cash.com/pneus/p/175-55-r55...,48,175/55 R15 77T,MILESTONE GREENSPORT,3,2024-03-23,MILESTONE,33209,D,...,70,Été,Tourisme,Non,33208,175,55,15,77,T
1,33213,https://www.carter-cash.com/pneus/p/175-55-r55...,59,175/55 R15 77T,NEXEN NBLUEHDPL,note inconnue,2024-03-23,NEXEN,33210,D,...,70,Été,Tourisme,Non,33209,175,55,15,77,T
2,33214,https://www.carter-cash.com/pneus/p/175-55-r55...,89,175/55 R15 77T,CONTINENTAL ALL SEASON CONTACT,note inconnue,2024-03-23,CONTINENTAL,33211,C,...,71,4 saisons,Tourisme,Non,33210,175,55,15,77,T
3,33216,https://www.carter-cash.com/pneus/p/175-55-r55...,76,175/55 R15 77T,FULDA ECOCONTROL,note inconnue,2024-03-23,FULDA,33213,D,...,70,Été,Tourisme,Non,33212,175,55,15,77,T
4,33217,https://www.carter-cash.com/pneus/p/175-55-r55...,79,175/55 R15 77T,HANKOOK KINERGY 4S H750,note inconnue,2024-03-23,HANKOOK,33214,D,...,71,4 saisons,Tourisme,Non,33213,175,55,15,77,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17156,62770,https://www.carter-cash.com/pneus/p/245-70-r70...,209,245/70 R17 114T,MICHELIN LATITUDE CROSS,note inconnue,2024-04-21,MICHELIN,62767,C,...,72,Été,4x4,Non,62766,245,70,17,0,T
17157,62771,https://www.carter-cash.com/pneus/p/245-70-r70...,147,245/70 R16 111T,CONTINENTAL CROSSCONTACT LX SPORT,note inconnue,2024-04-21,CONTINENTAL,62768,C,...,72,Été,4x4,Non,62767,245,70,16,0,T
17158,62773,https://www.carter-cash.com/pneus/p/245-70-r70...,144,245/70 R16 111S,BRIDGESTONE D689,note inconnue,2024-04-21,BRIDGESTONE,62770,C,...,72,Été,4x4,Non,62769,245,70,16,0,S
17159,62775,https://www.carter-cash.com/pneus/p/245-70-r70...,158,245/70 R16 111H,MICHELIN PRIMACY 4+,note inconnue,2024-04-21,MICHELIN,62772,B,...,71,Été,Tourisme,Non,62771,245,70,16,0,H


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17161 entries, 0 to 17160
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID_Produit          17161 non-null  int64 
 1   URL_Produit         17161 non-null  object
 2   Prix                17161 non-null  int64 
 3   Info_generale       17161 non-null  object
 4   Descriptif          17161 non-null  object
 5   Note                17161 non-null  object
 6   Date_scrap          17161 non-null  object
 7   Marque              17161 non-null  object
 8   ID_Caracteristique  17161 non-null  int64 
 9   Consommation        17161 non-null  object
 10  Indice_Pluie        17161 non-null  object
 11  Bruit               17161 non-null  int64 
 12  Saisonalite         17161 non-null  object
 13  Type_Vehicule       17161 non-null  object
 14  Runflat             17161 non-null  object
 15  ID_Dimension        17161 non-null  int64 
 16  Largeur             17

In [7]:
# Supprimer les colonnes 'ID_Caracteristique', 'ID_Dimension' et 'ID_Produit'
df = df.drop(['ID_Caracteristique', 'ID_Dimension', 'ID_Produit'], axis=1)
df


,URL_Produit,Prix,Info_generale,Descriptif,Note,Date_scrap,Marque,Consommation,Indice_Pluie,Bruit,Saisonalite,Type_Vehicule,Runflat,Largeur,Hauteur,Diametre,Charge,Vitesse
0,https://www.carter-cash.com/pneus/p/175-55-r55...,48,175/55 R15 77T,MILESTONE GREENSPORT,3,2024-03-23,MILESTONE,D,B,70,Été,Tourisme,Non,175,55,15,77,T
1,https://www.carter-cash.com/pneus/p/175-55-r55...,59,175/55 R15 77T,NEXEN NBLUEHDPL,note inconnue,2024-03-23,NEXEN,D,B,70,Été,Tourisme,Non,175,55,15,77,T
2,https://www.carter-cash.com/pneus/p/175-55-r55...,89,175/55 R15 77T,CONTINENTAL ALL SEASON CONTACT,note inconnue,2024-03-23,CONTINENTAL,C,B,71,4 saisons,Tourisme,Non,175,55,15,77,T
3,https://www.carter-cash.com/pneus/p/175-55-r55...,76,175/55 R15 77T,FULDA ECOCONTROL,note inconnue,2024-03-23,FULDA,D,C,70,Été,Tourisme,Non,175,55,15,77,T
4,https://www.carter-cash.com/pneus/p/175-55-r55...,79,175/55 R15 77T,HANKOOK KINERGY 4S H750,note inconnue,2024-03-23,HANKOOK,D,B,71,4 saisons,Tourisme,Non,175,55,15,77,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17156,https://www.carter-cash.com/pneus/p/245-70-r70...,209,245/70 R17 114T,MICHELIN LATITUDE CROSS,note inconnue,2024-04-21,MICHELIN,C,C,72,Été,4x4,Non,245,70,17,0,T
17157,https://www.carter-cash.com/pneus/p/245-70-r70...,147,245/70 R16 111T,CONTINENTAL CROSSCONTACT LX SPORT,note inconnue,2024-04-21,CONTINENTAL,C,C,72,Été,4x4,Non,245,70,16,0,T
17158,https://www.carter-cash.com/pneus/p/245-70-r70...,144,245/70 R16 111S,BRIDGESTONE D689,note inconnue,2024-04-21,BRIDGESTONE,C,D,72,Été,4x4,Non,245,70,16,0,S
17159,https://www.carter-cash.com/pneus/p/245-70-r70...,158,245/70 R16 111H,MICHELIN PRIMACY 4+,note inconnue,2024-04-21,MICHELIN,B,A,71,Été,Tourisme,Non,245,70,16,0,H


In [8]:
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import mean_squared_error ,  r2_score
# import numpy as np

# # Transformer "Date_scrap" en format datetime
# df['Date_scrap'] = pd.to_datetime(df['Date_scrap'])

# # Transformer "Date_scrap" en nombre de jours depuis la date la plus ancienne
# df['Date_scrap'] = (df['Date_scrap'] - df['Date_scrap'].min()).dt.days

# # Transformer "URL_Produit" en une forme numérique
# le = LabelEncoder()
# df['URL_Produit'] = le.fit_transform(df['URL_Produit'])

# # Sélectionner les caractéristiques et la cible
# X = df[['Date_scrap', 'URL_Produit']]
# y = df['Prix']

# # Diviser les données en ensemble d'entraînement et de test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Créer et entraîner le modèle
# model = LinearRegression()
# model.fit(X_train, y_train)

# # Prédire les prix
# y_pred = model.predict(X_test)

# # Calculer le R²
# r2 = r2_score(y_test, y_pred)

# print(f"R²: {r2}")


In [9]:
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import r2_score
# import pandas as pd

# # Convertir la colonne 'Date_scrap' en datetime
# df['Date_scrap'] = pd.to_datetime(df['Date_scrap'])

# # Créer une nouvelle colonne 'Day' pour représenter le jour de l'année
# df['Day'] = df['Date_scrap'].dt.dayofyear

# # Grouper les données par 'URL_Produit' et 'Day' et calculer la moyenne des prix
# df_grouped = df.groupby(['URL_Produit', 'Day'])['Prix'].mean().reset_index()

# # Préparer les données pour l'entraînement
# X = df_grouped[['URL_Produit', 'Day']]
# y = df_grouped['Prix']

# # Convertir 'URL_Produit' en valeurs numériques
# X['URL_Produit'] = X['URL_Produit'].astype('category').cat.codes

# # Diviser les données en ensembles d'entraînement et de test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Créer et entraîner le modèle
# model = LinearRegression()
# model.fit(X_train, y_train)

# # Faire des prédictions
# y_pred = model.predict(X_test)

# # Calculer le score R2
# r2 = r2_score(y_test, y_pred)

# print(f"Le score R2 du modèle est : {r2}")


In [10]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score
# import pandas as pd

# # Convertir la colonne 'Date_scrap' en datetime
# df['Date_scrap'] = pd.to_datetime(df['Date_scrap'])

# # Créer une nouvelle colonne 'Day' pour représenter le jour de l'année
# df['Day'] = df['Date_scrap'].dt.dayofyear

# # Grouper les données par 'URL_Produit' et 'Day' et calculer la moyenne des prix
# df_grouped = df.groupby(['URL_Produit', 'Day'])['Prix'].mean().reset_index()

# # Préparer les données pour l'entraînement
# X = df_grouped[['URL_Produit', 'Day']]
# y = df_grouped['Prix']

# # Convertir 'URL_Produit' en valeurs numériques
# X['URL_Produit'] = X['URL_Produit'].astype('category').cat.codes

# # Diviser les données en ensembles d'entraînement et de test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Créer et entraîner le modèle
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# # Faire des prédictions
# y_pred = model.predict(X_test)

# # Calculer le score R2
# r2 = r2_score(y_test, y_pred)

# print(f"Le score R2 du modèle est : {r2}")


In [11]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error



# Préparer les données pour l'entraînement
X = df.drop(columns=['Prix','URL_Produit','Info_generale','Date_scrap'])
y = df['Prix']

# Convertir les colonnes catégorielles en valeurs numériques
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category').cat.codes

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer et entraîner le modèle
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Faire des prédictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculer le score R2
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

mean_squared_error_train = mean_squared_error(y_train, y_pred_train)
mean_squared_error_test = mean_squared_error(y_test, y_pred_test)

mean_absolute_error_train = mean_absolute_error(y_train, y_pred_train)
mean_absolute_error_test = mean_absolute_error(y_test, y_pred_test)

# Log metrics in MLflow
mlflow.log_metric("r2_train", r2_train)
mlflow.log_metric("r2_test", r2_test)
mlflow.log_metric("mse_train", mean_squared_error_train)
mlflow.log_metric("mse_test", mean_squared_error_test)
mlflow.log_metric("mae_train", mean_absolute_error_train)
mlflow.log_metric("mae_test", mean_absolute_error_test)

print(f"Le score R2 du modèle sur les données d'entraînement est : {r2_train}")
print(f"Le score R2 du modèle sur les données de test est : {r2_test}")
print("------------------------------")
print(f"Le score MSE du modèle sur les données d'entraînement est : {mean_squared_error_train}")
print(f"Le score MSE du modèle sur les données de test est : {mean_squared_error_test}")
print("------------------------------")
print(f"Le score MAE du modèle sur les données d'entraînement est : {mean_absolute_error_train}")
print(f"Le score MAE du modèle sur les données de test est : {mean_absolute_error_test}")


Le score R2 du modèle sur les données d'entraînement est : 0.9919413246283255
Le score R2 du modèle sur les données de test est : 0.9784798406420636
------------------------------
Le score MSE du modèle sur les données d'entraînement est : 14.874386267495362
Le score MSE du modèle sur les données de test est : 40.02254025849503
------------------------------
Le score MAE du modèle sur les données d'entraînement est : 2.1175098133445873
Le score MAE du modèle sur les données de test est : 3.433708107692053


In [12]:
df

,URL_Produit,Prix,Info_generale,Descriptif,Note,Date_scrap,Marque,Consommation,Indice_Pluie,Bruit,Saisonalite,Type_Vehicule,Runflat,Largeur,Hauteur,Diametre,Charge,Vitesse
0,https://www.carter-cash.com/pneus/p/175-55-r55...,48,175/55 R15 77T,MILESTONE GREENSPORT,3,2024-03-23,MILESTONE,D,B,70,Été,Tourisme,Non,175,55,15,77,T
1,https://www.carter-cash.com/pneus/p/175-55-r55...,59,175/55 R15 77T,NEXEN NBLUEHDPL,note inconnue,2024-03-23,NEXEN,D,B,70,Été,Tourisme,Non,175,55,15,77,T
2,https://www.carter-cash.com/pneus/p/175-55-r55...,89,175/55 R15 77T,CONTINENTAL ALL SEASON CONTACT,note inconnue,2024-03-23,CONTINENTAL,C,B,71,4 saisons,Tourisme,Non,175,55,15,77,T
3,https://www.carter-cash.com/pneus/p/175-55-r55...,76,175/55 R15 77T,FULDA ECOCONTROL,note inconnue,2024-03-23,FULDA,D,C,70,Été,Tourisme,Non,175,55,15,77,T
4,https://www.carter-cash.com/pneus/p/175-55-r55...,79,175/55 R15 77T,HANKOOK KINERGY 4S H750,note inconnue,2024-03-23,HANKOOK,D,B,71,4 saisons,Tourisme,Non,175,55,15,77,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17156,https://www.carter-cash.com/pneus/p/245-70-r70...,209,245/70 R17 114T,MICHELIN LATITUDE CROSS,note inconnue,2024-04-21,MICHELIN,C,C,72,Été,4x4,Non,245,70,17,0,T
17157,https://www.carter-cash.com/pneus/p/245-70-r70...,147,245/70 R16 111T,CONTINENTAL CROSSCONTACT LX SPORT,note inconnue,2024-04-21,CONTINENTAL,C,C,72,Été,4x4,Non,245,70,16,0,T
17158,https://www.carter-cash.com/pneus/p/245-70-r70...,144,245/70 R16 111S,BRIDGESTONE D689,note inconnue,2024-04-21,BRIDGESTONE,C,D,72,Été,4x4,Non,245,70,16,0,S
17159,https://www.carter-cash.com/pneus/p/245-70-r70...,158,245/70 R16 111H,MICHELIN PRIMACY 4+,note inconnue,2024-04-21,MICHELIN,B,A,71,Été,Tourisme,Non,245,70,16,0,H


In [13]:

nouveau_produit = {
    'Descriptif': 'MILESTONE GREENSPORT',
    'Note': '3',
    'Marque': 'MILESTONE',
    'Consommation': 'D',
    'Indice_Pluie': 'B',
    'Bruit': 70,
    'Saisonalite': 'Été',
    'Type_Vehicule': 'Tourisme',
    'Runflat': 'Non',
    'Largeur': 175,
    'Hauteur': 55,
    'Diametre': 15,
    'Charge': 77,
    'Vitesse': 'T'
}

# Convertir le dictionnaire en dataframe
df_nouveau_produit = pd.DataFrame([nouveau_produit])


# Convertir les colonnes catégorielles en valeurs numériques
for col in df_nouveau_produit.select_dtypes(include=['object']).columns:
    df_nouveau_produit[col] = df_nouveau_produit[col].astype('category').cat.codes

# Faire une prédiction avec le modèle
prix_pred = model.predict(df_nouveau_produit)

print(f"Le prix prédit pour le nouveau produit est : {prix_pred[0]}")


Le prix prédit pour le nouveau produit est : 73.38116666666667


In [14]:


# # Définir l'URL du produit et la date de scraping
# url_produit = 'https://www.carter-cash.com/pneus/p/205-45-r17-88Y-361024-UNI08'
# date_scrap = '2024-04-06'

# # Convertir la date de scraping en datetime et obtenir le jour de l'année
# date_scrap = pd.to_datetime(date_scrap)
# day = date_scrap.dayofyear

# # Obtenir la liste des URL de produits
# url_list = df['URL_Produit'].astype('category').cat.categories.tolist()

# # Vérifier si l'URL du produit est dans la liste
# if url_produit in url_list:
#     # Obtenir le code numérique pour l'URL du produit
#     url_code = url_list.index(url_produit)

#     # Préparer les données pour la prédiction
#     X_new = pd.DataFrame({'URL_Produit': [url_code], 'Day': [day]})

#     # Faire la prédiction
#     prix_pred = model.predict(X_new)

#     print(f"Le prix prédit pour le produit {url_produit} à la date {date_scrap} est : {prix_pred[0]}")
# else:
#     print(f"L'URL du produit {url_produit} n'est pas dans la liste des URL de produits.")


In [15]:
# import pickle

# # Sauvegarder le modèle dans un fichier pickle
# with open('model.pkl', 'wb') as file:
#     pickle.dump(model, file)

# print("Le modèle a été sauvegardé dans le fichier 'model.pkl'.")
